<h1>Covid data for all  states in India </h1>

<h3> Library required for this project </h3>

In [110]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import chart_studio.plotly as py
import seaborn as sns
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
from datetime import date


from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected = True)
cf.go_offline()

<h3> Web Scrapping the Arogya Setu Site </h3>
<h6> The offical data for covid cases show by indian government </h6>

In [111]:
database = 'covidindi.db' # pleasse add the database name as per your requirement

In [112]:
url = 'https://www.mygov.in/corona-data/covid19-statewise-status/'
today = date.today()
response = requests.get(url)
html_data = response.text
data = bs(html_data,'html.parser')


<h6> Storing All States Name in form of array </h6>


In [113]:
x = data.find_all(class_ = "field field-name-field-select-state field-type-list-text field-label-above")
state = []
for i in x:
    state.append(i.find(class_ = 'field-item even').text) 

<h6> Storing all active cases in form of array</h6>

In [114]:
x = data.find_all(class_ ="field field-name-field-total-confirmed-indians field-type-number-integer field-label-above")
confirmed_cases = []
for i in x:
    confirmed_cases.append(i.find(class_ = 'field-item even').text)

<h6> Storing all Cured Cases in form of array </h6>

In [115]:
x = data.find_all(class_ ="field field-name-field-cured field-type-number-integer field-label-above")
cured_cases = []
for i in x:
    cured_cases.append(i.find(class_ = 'field-item even').text)

<h6> Storing all death in form of array </h6>

In [116]:
x = data.find_all(class_ ="field field-name-field-deaths field-type-number-integer field-label-above")
death_cases = []
for i in x:
    death_cases.append(i.find(class_ = 'field-item even').text)

<h6> Making a DataFrame all the webcrapped data by help pandas</h6>

In [117]:
india_da = {'State': state,
           'Total Cases':confirmed_cases,
           'Cured': cured_cases,
           'Death':death_cases}
covid_india = pd.DataFrame(india_da) # making data frame as covid_india
change_type = {
    'Total Cases':int,
    'Cured':int,
    'Death': int
}
covid_india = covid_india.astype(change_type)
# additonally adding the active cases By [active cases = Total cases -(cured cases + death cases)]
covid_india["Active Cases"] =  covid_india['Total Cases'] -(covid_india['Cured']+ covid_india['Death'])
S_NO = np.arange(1,len(state)+1,1) # adding a serial no.
covid_india.insert(loc = 0,column = 'S.No.',value = S_NO)


<h1> Task 1</h1>
<h6> Saving the scrapped data is csv format file in same directory as project is running </h6>

In [118]:
covid_india.to_csv(f'covid_india{today}.csv',index = False) 

<h6>Setting S.NO. as my index in dataframe</h6>

In [119]:
covid_india = covid_india.set_index('S.No.')

<h1> Using SqLite database to save my webscrapped Data </h1>

In [120]:
import sqlite3 
con = sqlite3.connect(database) # setting up and making a connection with my database

<h1> Task 2 </h1>
<h6> Saving my covid_india dataframe in database covidindia.db with table name as covid_India </h6>

In [121]:
covid_india.to_sql(name = 'covid_India',con = con)

In [122]:
pd.read_sql('select * from covid_India',con) # fetching up the table from my database in order to use in future  as per my convinent

,S.No.,State,Total Cases,Cured,Death,Active Cases
0,1,Andaman and Nicobar,7889,7628,129,132
1,2,Andhra Pradesh,2079763,2062290,14501,2972
2,3,Arunachal Pradesh,55462,55053,282,127
3,4,Assam,624499,613373,6178,4948
4,5,Bihar,735852,715262,12100,8490
5,6,Chandigarh,67214,64811,1080,1323
6,7,Chhattisgarh,1017356,994063,13609,9684
7,8,Dadra and Nagar Haveli and Daman and Diu,10752,10688,4,60
8,9,Delhi,1506798,1441789,25136,39873
9,10,Goa,186198,176737,3530,5931


<h6> Top 5 most infected city in india</h6>

In [123]:
df = covid_india.nlargest(5,["Total Cases"])
fig = px.bar(df,x ='State',y = 'Total Cases', color = 'State', text = 'Total Cases',
            title = 'Top 5 Infected States in India',
            labels = {'State': 'Major Effected States'}).update_layout(xaxis_tickangle = 45,uniformtext_minsize = 8)
fig.update_traces(texttemplate = '%{text:.2s}',textposition = 'outside')

<h6> Top 5 States having most death due covid</h6>

In [124]:
df = covid_india.nlargest(5,["Death"])
fig = px.bar(df,x ='State',y = 'Death', color = 'State', text = 'Death',
            title = 'Top 5 States in India where covid is Life Threatening',
            labels = {'State': 'Major Effected States'}).update_layout(xaxis_tickangle = 45,uniformtext_minsize = 8)
fig.update_traces(texttemplate = '%{text:.2s}',textposition = 'outside')

<h6>Current Status of covid in India</h6>

In [125]:
df = covid_india.copy()
fig = px.bar(df,x ='State',y = 'Active Cases', color = 'State', text = 'Active Cases',
            title = f'Current Status of covid in India as on {today}',
            labels = {'State': 'All States'}).update_layout(xaxis_tickangle = 45,uniformtext_minsize = 8)
fig.update_traces(texttemplate = '%{text:.2s}',textposition = 'outside')

In [ ]:

df['Recovery %'] = (df['Cured']/df['Total Cases'])*100
df['Death %'] = (df['Death']/df['Total Cases'])*100
df['Total_cases_%_wrt_india'] = (df['Total Cases']/sum(df['Total Cases']))*100
df['Death_%_wrt_india'] = (df['Death']/sum(df['Death']))*100
df.nlargest(5,'Recovery %')



In [ ]:
fig = px.bar(df,x ='State',y = 'Active Cases', color = 'State', text = 'Active Cases',
            title = f'Current Status of covid in India as on {today}',
            labels = {'State': 'All States'}).update_layout(xaxis_tickangle = 45,uniformtext_minsize = 8)
fig.update_traces(texttemplate = '%{text:.2s}',textposition = 'outside')

<h6>Covid effect in india</h6>

In [ ]:

px.pie(df, values = 'Total Cases', names = 'State',
      title = 'Covid effect in India',
      color_discrete_sequence = px.colors.sequential.RdBu)

In [ ]:
px.pie(df, values = 'Active Cases', names = 'State',
      title = 'Active Covid Status in India',
      color_discrete_sequence = px.colors.sequential.RdBu)

In [ ]:
px.pie(df, values = 'Death', names = 'State',
      title = 'Death by covid in India',
      color_discrete_sequence = px.colors.sequential.RdBu)